In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
data_dir = '/kaggle/input/preprocessed-brain-mri-images/brain-tumor/processed-images'
batch_size = 32
img_height = 224
img_width = 224

In [3]:
data = []
labels = []

for subdir in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir)

    if os.path.isdir(subdir_path):
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)

            if file_path.endswith('.jpg') or file_path.endswith('.png'):
                data.append(file_path)
                labels.append(subdir)

In [4]:
print(len(data))
print(len(labels))

12256
12256


In [5]:
df = pd.DataFrame({'filename': data, 'class': labels})

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)

In [7]:
datagen = keras.preprocessing.image.ImageDataGenerator()

In [8]:
def initial_training(img_height, img_width, batch_size, datagen, train_df, test_df, dropout_rate):
    
    train_data = datagen.flow_from_dataframe(
        train_df,
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )
    test_data = datagen.flow_from_dataframe(
        test_df,
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    base_model = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(img_height, img_width,3)
    )
    base_model.trainable = False
    
    image_batch, label_batch = next(iter(train_data))
    feature_batch = base_model(image_batch)
    
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_batch_average = global_average_layer(feature_batch)
    
    prediction_layer = keras.Sequential( [
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(3)
    ])
    prediction_batch = prediction_layer(feature_batch_average)
    
    preprocess_input = tf.keras.applications.resnet50.preprocess_input
    
    inputs = tf.keras.Input(shape=(img_height, img_width,3) )
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(dropout_rate+0.1)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    
    base_learning_rate = 0.0001
    model.compile(optimizer=tf.keras.optimizers.experimental.Adam(learning_rate=base_learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    initial_epochs = 7
    history = model.fit(train_data,
                        epochs=initial_epochs,
                        validation_data=test_data)
    return model

In [9]:
def create_model(model, fine_tune_start):
    base_model = model.get_layer(index=3)
    base_model.trainable = True
    
    fine_tune_at = fine_tune_start

    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable = False
    base_learning_rate= 0.0001
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
                  metrics=['accuracy'])
    return model

In [10]:
def fine_tuning(img_height, img_width, batch_size, datagen, train_df, test_df, model, fine_tune_start, acc_per_comb):
    fine_tuning_epoch = 12
    
    train_data = datagen.flow_from_dataframe(
        train_df,
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )
    test_data = datagen.flow_from_dataframe(
        test_df,
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    fine_tuning_model = create_model(model, fine_tune_start)
    
    history = fine_tuning_model.fit(train_data, validation_data=test_data, epochs=fine_tuning_epoch)
    
    scores = fine_tuning_model.evaluate(test_data)
    acc_per_comb.append(scores[1])
    return fine_tuning_model

In [11]:
fine_tune_start_list = [100, 120, 140]
dropout_rate_list = [0.2, 0.3, 0.4]
combination = []
acc_per_comb = []

In [12]:
def grid_search(fine_tune_start, dropout_rate, acc_per_comb):
    model = initial_training(img_height, img_width, batch_size, datagen, train_df, test_df, dropout_rate)
    fine_tuned_model = fine_tuning(img_height, img_width, batch_size, datagen, train_df, test_df, model, fine_tune_start, acc_per_comb)
    return

In [13]:
for fine_tune_start in fine_tune_start_list:
    for dropout_rate in dropout_rate_list:
        combination.append([fine_tune_start, dropout_rate])
        grid_search(fine_tune_start, dropout_rate, acc_per_comb)

Found 9804 validated image filenames belonging to 3 classes.
Found 2452 validated image filenames belonging to 3 classes.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/7
307/307 [==============================] - 63s 191ms/step - loss: 0.7231 - accuracy: 0.7119 - val_loss: 0.3914 - val_accuracy: 0.8454
Epoch 2/7
307/307 [==============================] - 33s 106ms/step - loss: 0.4599 - accuracy: 0.8161 - val_loss: 0.3081 - val_accuracy: 0.8764
Epoch 3/7
307/307 [==============================] - 31s 102ms/step - loss: 0.3938 - accuracy: 0.8457 - val_loss: 0.2856 - val_accuracy: 0.8817
Epoch 4/7
307/307 [==============================] - 30s 98ms/step - loss: 0.3539 - accuracy: 0.8589 - val_loss: 0.2878 - val_accuracy: 0.8793
Epoch 5/7
307/307 [==============================] - 31s 100ms/step - loss: 0.3537 - accuracy: 0.8629 - val_loss: 0.2583 - val_accuracy: 0.8993
Epoch 6/7
307/307 [==============================] - 31s 99ms/step - loss: 0.3221 - accuracy: 

In [15]:
print(combination)
print(acc_per_comb)

[[100, 0.2], [100, 0.3], [100, 0.4], [120, 0.2], [120, 0.3], [120, 0.4], [140, 0.2], [140, 0.3], [140, 0.4]]
[0.9738988876342773, 0.9698205590248108, 0.9657422304153442, 0.9698205590248108, 0.9714518785476685, 0.970228374004364, 0.9526916742324829, 0.9620717763900757, 0.9571778178215027]


In [17]:
best_comb_index = 0
best_acc = 0
for i in range(len(acc_per_comb)):
    if(i == 0):
        best_acc = acc_per_comb[i]
    elif(best_acc < acc_per_comb[i]):
        best_acc = acc_per_comb[i]
        best_comb_index = i

In [18]:
print(combination[best_comb_index])

[100, 0.2]
